In [181]:
import sys
sys.path.append('../')
os.chdir('C:/Users/david/Dropbox/mobilier/crypto/')
from ftx_utilities import *
%matplotlib inline

In [189]:
concentration_limit=0.5
slippage_override=2e-4
sharpe=lambda x: np.mean(x)/np.std(x)
sortino=lambda x: np.mean(x)/np.std(x[x<0])
drawdown=lambda x: (x.cumsum()-x.cumsum().cummax()).cummin().loc[x.index[-1]]

In [196]:
brownian=pd.Series(np.random.exponential(size=100,scale=.1))
drawdown_=pd.Series(name=str(sharpe(brownian))+' / '+str(sortino(brownian)),data=[drawdown(brownian)]*100)
pd.concat([brownian.cumsum(),drawdown_],axis=1).iplot()

In [140]:
# basics
def read_it(filename):
    df=pd.read_pickle(filename)
    df=df[
        (df['concentration_limit']==concentration_limit)
        &(df['slippage_override']==slippage_override)]
    #df[df.select_dtypes(include=['number']).columns] *= 100/1e5
    df['totalPnL']=(df['RealizedCarry']+df['transactionCost'])
    return df.drop(['concentration_limit','slippage_override'],axis=1)

## benchmark

In [197]:
# read benchmark
df=read_it('archived data/benchmark_blind.pickle')
mdf=df.set_index(['name','time']) \
        .unstack('name') \
            .reorder_levels([1,0],axis=1)

KeyError: 'Level signal_horizon not found'

In [34]:
perfs=df.groupby(by=['name'])['absWeight'].describe().sort_values(by='mean',key=abs,ascending=False).head(10)
mains=df[df['name'].isin(list(perfs.index)+['total','USD'])]

In [36]:
stats=mains.groupby(by=['name'])[['RealizedCarry','transactionCost','trackingError','optimalWeight']].agg(['mean','std'])*100/1e5
stats[('relError','')]=np.sqrt((np.power(stats[('trackingError','mean')],2)+np.power(stats[('trackingError','std')],2))/(np.power(stats[('RealizedCarry','mean')],2)+np.power(stats[('RealizedCarry','std')],2)))
stats.sort_values(by=('transactionCost','mean'))

field      RealizedCarry            transactionCost             trackingError  \
                    mean        std            mean         std          mean   
name                                                                            
total         105.409644  48.316056     -504.508569  349.185739    -14.234859   
CEL-PERP        4.697713   8.351736      -38.882663   55.403079     -2.333005   
BNT-PERP        7.526665  11.096281      -35.475382   45.639663     -1.687329   
ALPHA-PERP      6.612391   9.202989      -23.259970   38.014915     -0.639471   
OKB-PERP       14.022361  16.810620      -22.723497   42.302906     -1.102332   
RUNE-PERP       6.124588   9.752879      -21.810663   35.413633     -0.649864   
GRT-PERP        8.068580  10.816846      -21.233930   33.993033     -0.543631   
SUSHI-PERP      4.693480   7.307900      -20.254219   31.841700     -0.368824   
MKR-PERP        6.922685   8.796582      -19.791104   32.166179     -0.439694   
USD            -6.690893  14.555804        0.000000    0.000000     -0.250589   

field                 optimalWeight                relError  
                  std          mean           std            
name                                                         
total       31.816274    100.000000  3.458312e-14  0.300594  
CEL-PERP     8.285114      3.728075  2.754214e+01  0.898255  
BNT-PERP     7.547994    -18.249089  1.929092e+01  0.576836  
ALPHA-PERP   4.272451     18.398399  2.439167e+01  0.381218  
OKB-PERP     8.853247    -11.250703  3.568008e+01  0.407544  
RUNE-PERP    4.534670      8.367129  2.467545e+01  0.397778  
GRT-PERP     5.081284     -9.514408  2.878132e+01  0.378689  
SUSHI-PERP   2.448457     17.471978  2.086610e+01  0.285089  
MKR-PERP     3.059837      2.490145  2.983602e+01  0.276156  
USD          6.649631    -10.230468  1.835580e+02  0.415378

In [37]:
(mains.pivot_table(index='time',columns='name',values='totalPnL',aggfunc='mean')/24/365.25).iplot()

## runs

In [254]:
# read full run
df=read_it('archived data/full_run.pickle').set_index('time')

# compute tracking error = <real-exp>/<real^2>
(r,e)=(df['RealizedCarry'],df['ExpectedCarry'])
df['trackingError']=np.sqrt(((r-e)*(r-e)).ewm(halflife=timedelta(days=7),times=r.index).mean()/(e*e).ewm(halflife=timedelta(days=7),times=r.index).mean())

mdf=df.reset_index().set_index(['signal_horizon','holding_period','name','time']) \
        .unstack(['signal_horizon','holding_period','name']) \
            .reorder_levels([1,2,3,0],axis=1)
total=mdf.xs(('totalPnL','total'),level=('field','name'),axis=1)*100/1e5
txcost=mdf.xs(('transactionCost','total'),level=('field','name'),axis=1)*100/1e5

In [255]:
# sharpe is bad
total.apply(lambda f: sharpe(f.dropna()),axis=0).unstack()

holding_period,0 days 06:00:00,0 days 12:00:00,1 days 00:00:00,2 days 00:00:00,3 days 00:00:00,4 days 00:00:00,5 days 00:00:00
signal_horizon,,,,,,,
0 days 12:00:00,-0.058615,0.491320,NaN,NaN,NaN,NaN,NaN
1 days 00:00:00,-0.058208,0.294010,0.924270,NaN,NaN,NaN,NaN
2 days 00:00:00,-0.054421,0.429306,0.997867,1.173736,NaN,NaN,NaN
3 days 00:00:00,-0.061333,0.305493,0.902892,1.118022,1.286722,NaN,NaN
4 days 00:00:00,-0.059642,0.260735,0.846793,1.094914,1.260790,1.312444,NaN
5 days 00:00:00,-0.047017,0.178943,0.702489,1.190681,1.167614,1.341646,1.551580
7 days 00:00:00,-0.059483,0.102340,0.884217,1.148275,1.197592,1.276588,1.253307
10 days 00:00:00,-0.058636,-0.083802,0.824453,1.028955,1.016925,1.120752,1.202375
30 days 00:00:00,-0.050424,-0.098218,0.298392,0.590868,0.707584,0.875428,0.853092


In [256]:
# sortino is great
total.apply(lambda f: sortino(f.dropna()),axis=0).unstack()

C:\Users\david\AppData\Local\Temp/ipykernel_32376/14136895.py:4: RuntimeWarning:

divide by zero encountered in double_scalars



holding_period,0 days 06:00:00,0 days 12:00:00,1 days 00:00:00,2 days 00:00:00,3 days 00:00:00,4 days 00:00:00,5 days 00:00:00
signal_horizon,,,,,,,
0 days 12:00:00,-0.027135,0.523267,NaN,NaN,NaN,NaN,NaN
1 days 00:00:00,-0.023707,0.320548,1.411857,NaN,NaN,NaN,NaN
2 days 00:00:00,-0.021737,0.463662,1.981812,3.319196,NaN,NaN,NaN
3 days 00:00:00,-0.024642,0.232153,1.485113,3.239701,inf,NaN,NaN
4 days 00:00:00,-0.024294,0.165247,1.190225,3.241436,inf,29.054783,NaN
5 days 00:00:00,-0.022217,0.134381,1.211995,4.692626,2.943249,14.781833,inf
7 days 00:00:00,-0.031417,0.064155,2.374393,9.885021,inf,inf,NaN
10 days 00:00:00,-0.029024,-0.051797,2.213880,3.723148,5.822618,44.228750,14.343324
30 days 00:00:00,-0.025771,-0.056541,1.110627,2.220044,1.972808,4.927057,4.458588


In [257]:
# return is good
total.apply(lambda f: f.dropna().mean(),axis=0).unstack()

holding_period,0 days 06:00:00,0 days 12:00:00,1 days 00:00:00,2 days 00:00:00,3 days 00:00:00,4 days 00:00:00,5 days 00:00:00
signal_horizon,,,,,,,
0 days 12:00:00,-1.474008,14.288459,NaN,NaN,NaN,NaN,NaN
1 days 00:00:00,-1.101679,10.372719,40.698082,NaN,NaN,NaN,NaN
2 days 00:00:00,-0.697315,16.207606,42.860514,47.567107,NaN,NaN,NaN
3 days 00:00:00,-1.003769,8.684340,41.256386,46.800902,45.778020,NaN,NaN
4 days 00:00:00,-1.342554,7.517959,39.913549,45.424164,46.832071,47.693318,NaN
5 days 00:00:00,-1.044049,5.111861,33.101022,46.879916,42.185828,43.644887,40.949906
7 days 00:00:00,-0.799141,1.108357,37.383488,45.557827,43.985288,43.762704,40.035766
10 days 00:00:00,-0.589249,-0.628963,34.217766,42.460240,39.419152,39.207825,41.196390
30 days 00:00:00,-0.261456,-0.344476,12.176273,21.474507,21.251036,26.696884,24.570942


In [258]:
# drawdown is great
total.apply(lambda f: drawdown(f.dropna()),axis=0).unstack()*100/1e5

holding_period,0 days 06:00:00,0 days 12:00:00,1 days 00:00:00,2 days 00:00:00,3 days 00:00:00,4 days 00:00:00,5 days 00:00:00
signal_horizon,,,,,,,
0 days 12:00:00,-0.057411,-0.129161,NaN,NaN,NaN,NaN,NaN
1 days 00:00:00,-0.039523,-0.228705,-0.171751,NaN,NaN,NaN,NaN
2 days 00:00:00,-0.016773,-0.170500,-0.108548,-0.059572,NaN,NaN,NaN
3 days 00:00:00,-0.047709,-0.167526,-0.087300,-0.017472,-0.017107,NaN,NaN
4 days 00:00:00,-0.049605,-0.196519,-0.081449,-0.048484,-0.007983,-0.007112,NaN
5 days 00:00:00,-0.064112,-0.175991,-0.088402,-0.022577,-0.030746,-0.010342,-0.001248
7 days 00:00:00,-0.025634,-0.048166,-0.091254,-0.016584,-0.012549,-0.001047,0.000000
10 days 00:00:00,-0.011592,-0.078324,-0.107923,-0.028940,-0.018394,-0.005421,-0.010881
30 days 00:00:00,-0.009837,-0.010798,-0.178183,-0.093025,-0.063610,-0.050543,-0.034553


## one run summary

In [259]:
total=df[(df['signal_horizon']==timedelta(days=5))
         &(df['holding_period']==timedelta(days=2))
         &(df['name']=='total')]
total.loc[total.index.min(),'transactionCost']=0 # no intital tx cost..
(total[['RealizedCarry','transactionCost']].cumsum()*2/365.25/1e5).iplot(title='cumulative excess return in %')

In [260]:
total[['RealizedCarry','ExpectedCarry','trackingError']]

field,RealizedCarry,ExpectedCarry,trackingError
time,,,
2021-07-08,50240.400147,66031.782396,0.890510
2021-07-10,44140.399353,67625.777723,0.890509
2021-07-12,62544.337416,67402.729141,0.890507
2021-07-14,34622.275172,67726.962044,0.890506
2021-07-16,44839.276880,67501.980306,0.890504
2021-07-18,42992.117657,49849.604432,0.890503
2021-07-20,28411.220672,46530.193465,0.890502
2021-07-22,32289.658996,55721.535978,0.890500
2021-07-24,-15705.061836,55144.382724,0.890506
